In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Import the necessary python packages.**

In [2]:
import sys
import math
import time
import torch
import pickle
import random
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as utils
from torch.autograd import Variable
from torch.nn.parameter import Parameter
from sklearn.metrics import mean_absolute_error, mean_squared_error, accuracy_score

## Data Processing

**Import Data**.



``` Main matrix ``` with 410 stops, starting from ```2021-09-13 05:00:00``` to ```2021-10-10 23:00:00```.

```A``` is the corresponding adjacency matrix, and ```BNP``` is the corresponding  matrix.



In [3]:
main_df = pd.read_pickle("/content/drive/MyDrive/pfe/main_matrix_5_23.pkl")
with open('/content/drive/MyDrive/pfe/A_5_23','rb') as f:
     A = pickle.load(f)
BNP = np.load('FFR.npy')

**Data preparation**

```ConcatDataset``` class help to consider 3 types of datasets simulatnously (recent, daily-periodic and weekly-periodic data).

In [4]:
class ConcatDataset(torch.utils.data.Dataset):
    def __init__(self, *datasets):
        self.datasets = datasets

    def __getitem__(self, i):
        return tuple(d[i %len(d)] for d in self.datasets)

    def __len__(self):
        return max(len(d) for d in self.datasets)

In [5]:
def PrepareDataset(main_passenger_matrix, BATCH_SIZE = 20,  pred_len = 1, train_propotion = 0.7, valid_propotion = 0.2):

    time_len = main_passenger_matrix.shape[0]
    
    max_load = main_passenger_matrix.max().max()
    main_passenger_matrix =  main_passenger_matrix / max_load
    

    # Weekly-periodic data preparation
    pass_sequences, pass_labels = [], []
    
    # Number of historical sequence
    seq_len = 2

    for i in range(time_len - 109*7*(seq_len + pred_len)):
        pass_sequences.append(main_passenger_matrix.iloc[[i+109*7*j for j in range(seq_len)]].values)  #109 is the number of steps between two consecutive days
        pass_labels.append(main_passenger_matrix.iloc[[i+109*7*(seq_len+j) for j in range(pred_len)]].values)
        
        

    pass_sequences, pass_labels = np.asarray(pass_sequences), np.asarray(pass_labels)
    
    sample_size = pass_sequences.shape[0]
    
    
    train_index = int(np.floor(sample_size * train_propotion))  
    valid_index = int(np.floor(sample_size * ( train_propotion + valid_propotion)))
    
    # randomize the order of sequences
    
    c = list(zip(pass_sequences, pass_labels))
    random.shuffle(c)
    pass_sequences, pass_labels = zip(*c)
   
    train_data, train_label = pass_sequences[:train_index], pass_labels[:train_index]    
    valid_data, valid_label = pass_sequences[train_index:valid_index], pass_labels[train_index:valid_index] 
    test_data, test_label = pass_sequences[valid_index:], pass_labels[valid_index:]
    
    
    train_data, train_label = torch.Tensor(train_data), torch.Tensor(train_label) 
    valid_data, valid_label = torch.Tensor(valid_data), torch.Tensor(valid_label) 
    test_data, test_label = torch.Tensor(test_data), torch.Tensor(test_label)
    
   
    train_dataset_w = utils.TensorDataset(train_data, train_label)
    valid_dataset_w = utils.TensorDataset(valid_data, valid_label)
    test_dataset_w = utils.TensorDataset(test_data, test_label)


    # Daily-periodic data preparation

    seq_len = 4
    pass_sequences, pass_labels = [], []
    
    # Specify the time range from which to get the data
    maind = main_passenger_matrix.loc['2021-09-23 05:00:00':'2021-10-04 23:00:00']

    time_len = maind.shape[0]

    for i in range(time_len - 109*(seq_len + pred_len)):
        pass_sequences.append(maind.iloc[[i+109*j for j in range(seq_len)]].values)
        pass_labels.append(maind.iloc[[i+109*(seq_len+j) for j in range(pred_len)]].values)

        
 
    pass_sequences, pass_labels = np.asarray(pass_sequences), np.asarray(pass_labels)
    
    sample_size = pass_sequences.shape[0]   
    index = np.arange(sample_size, dtype = int) 
    np.random.shuffle(index)
    
    train_index = int(np.floor(sample_size * train_propotion))  
    valid_index = int(np.floor(sample_size * ( train_propotion + valid_propotion)))
    
    # randomize the order of sequences
    
    c = list(zip(pass_sequences, pass_labels))
    random.shuffle(c)
    pass_sequences, pass_labels = zip(*c)
   
    train_data, train_label = pass_sequences[:train_index], pass_labels[:train_index]    
    valid_data, valid_label = pass_sequences[train_index:valid_index], pass_labels[train_index:valid_index] 
    test_data, test_label = pass_sequences[valid_index:], pass_labels[valid_index:]
    
    
    train_data, train_label = torch.Tensor(train_data), torch.Tensor(train_label) 
    valid_data, valid_label = torch.Tensor(valid_data), torch.Tensor(valid_label) 
    test_data, test_label = torch.Tensor(test_data), torch.Tensor(test_label)
    
   
    train_dataset_d = utils.TensorDataset(train_data, train_label)
    valid_dataset_d = utils.TensorDataset(valid_data, valid_label)
    test_dataset_d = utils.TensorDataset(test_data, test_label)

    # Recent data preparation

    seq_len = 10
    pass_sequences, pass_labels = [], []
    
    # Specify the time range from which to get the data
    mainr = main_passenger_matrix.loc['2021-09-26 21:30:00':'2021-10-04 05:00:00']

    time_len = mainr.shape[0]
    for i in range(time_len - (seq_len + pred_len)):
        pass_sequences.append(mainr.iloc[[i+ j for j in range(seq_len)]].values)  
        pass_labels.append(mainr.iloc[[i+(seq_len+j) for j in range(pred_len)]].values)
       
    
    pass_sequences, pass_labels = np.asarray(pass_sequences), np.asarray(pass_labels)
    
    sample_size = pass_sequences.shape[0]   
    index = np.arange(sample_size, dtype = int) 
    np.random.shuffle(index) 
    
    train_index = int(np.floor(sample_size * train_propotion))  
    valid_index = int(np.floor(sample_size * ( train_propotion + valid_propotion)))
    
    # randomize the order of sequences
    
    c = list(zip(pass_sequences, pass_labels))
    random.shuffle(c)
    pass_sequences, pass_labels = zip(*c)
    
       
    train_data, train_label = pass_sequences[:train_index], pass_labels[:train_index]    
    valid_data, valid_label = pass_sequences[train_index:valid_index], pass_labels[train_index:valid_index] 
    test_data, test_label = pass_sequences[valid_index:], pass_labels[valid_index:]    
    
    
    train_data, train_label = torch.Tensor(train_data), torch.Tensor(train_label) 
    valid_data, valid_label = torch.Tensor(valid_data), torch.Tensor(valid_label) 
    test_data, test_label = torch.Tensor(test_data), torch.Tensor(test_label)
    
   
    train_dataset_r = utils.TensorDataset(train_data, train_label)
    valid_dataset_r = utils.TensorDataset(valid_data, valid_label)
    test_dataset_r = utils.TensorDataset(test_data, test_label)



    # All the data in one dataloader
    
    train_loader = torch.utils.data.DataLoader(
                ConcatDataset(train_dataset_r, train_dataset_d,train_dataset_w),
                batch_size=BATCH_SIZE, shuffle=True, drop_last = True)
    
    valid_loader = torch.utils.data.DataLoader(
                ConcatDataset(valid_dataset_r, valid_dataset_d,valid_dataset_w),
                batch_size=BATCH_SIZE, shuffle=True, drop_last =True)
    
    test_loader = torch.utils.data.DataLoader(
                ConcatDataset(test_dataset_r, test_dataset_d,test_dataset_w),
                batch_size=BATCH_SIZE, shuffle=True, drop_last= True)
    
    return train_loader, valid_loader, test_loader, max_load

## Model Architecture

Temporal Gated Dilated Convolution (Temporal-GDCN) Layer 

In [6]:
class GDCN(nn.Module):

  def __init__(self, in_dim=1, blocks=2, layers = 2 , residual_channels=1, dilation_channels=1, kernel_size=2):
    super(GDCN, self).__init__()
    self.blocks = blocks
    self.layers = layers
    self.filter_convs = nn.ModuleList()
    self.gate_convs = nn.ModuleList()
    self.start_conv = nn.Conv2d(in_channels=in_dim, out_channels=residual_channels, kernel_size=(1,1), padding='same')
  
    for b in range(blocks):
      new_dilation = 1
      for i in range(layers):
        #dialated convolutions
        self.filter_convs.append(nn.Conv2d(in_channels=residual_channels, out_channels=dilation_channels, 
                                           kernel_size=(1,kernel_size), dilation=new_dilation, padding='same'))
        self.gate_convs.append(nn.Conv2d(in_channels=residual_channels, out_channels=dilation_channels,
                                         kernel_size=(1, kernel_size), dilation=new_dilation, padding='same'))

  def forward(self, input):
      
    x = torch.unsqueeze(torch.unsqueeze(input,0),0)
    x = self.start_conv(x)

    for i in range(self.blocks*self.layers):
      residual = x

      # dilated convolution
      filter = self.filter_convs[i](residual)
      filter = torch.tanh(filter)
           
      gate = self.gate_convs[i](residual)
      gate = torch.sigmoid(gate)

      x = filter * gate

    x = torch.squeeze(x,1)
    x = torch.permute(x, (0,2,1))
    x = torch.squeeze(x)

    return x

Spatial Graph Attention Convolution (Spatial-GACN) Layer


*   Graph Attention Network (GAT)
*   Graph Concolution Network (GCN)



In [19]:
class GAT(nn.Module):
  def __init__(self, in_features = 200, out_features =200, dropout=0.3):
    super(GAT, self).__init__()
    self.dropout = dropout
    self.in_features = in_features
    self.out_features = out_features
    self.W = nn.Parameter(torch.empty(size=(in_features, out_features)))
    nn.init.xavier_uniform_(self.W.data, gain=1.414)
    self.a = nn.Parameter(torch.empty(size=(2*out_features, 1)))
    nn.init.xavier_uniform_(self.a.data, gain=1.414)
    self.leakyrelu = nn.LeakyReLU(0.2)
  
  def forward(self, h, adj):
    # adj is the adjacency matrix

    Wh = torch.matmul(h, self.W)
    e = self._prepare_attentional_mechanism_input(Wh)

    zero_vec = -9e15*torch.ones_like(e)
    attention = torch.where(adj > 0, e, zero_vec)
    attention = adj
    attention = F.softmax(attention, dim=1)
    attention = F.dropout(attention, self.dropout, training=self.training)
    h_prime = torch.matmul(attention.float(), Wh)

    return F.elu(h_prime)
  
  def _prepare_attentional_mechanism_input(self, Wh):
        
    Wh1 = torch.matmul(Wh, self.a[:self.out_features, :])
    Wh2 = torch.matmul(Wh, self.a[self.out_features:, :])
    
    # broadcast add
    e = Wh1 + Wh2.T
    return self.leakyrelu(e)


In [20]:
class FilterLinear(nn.Module):
    def __init__(self, in_features, out_features, filter_square_matrix, bias=True):
        '''
        filter_square_matrix : filter square matrix, whose each elements is 0 or 1.
        '''
        super(FilterLinear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        
        use_gpu = torch.cuda.is_available()
        self.filter_square_matrix = None
        if use_gpu:
            self.filter_square_matrix = Variable(filter_square_matrix.cuda(), requires_grad=False)
        else:
            self.filter_square_matrix = Variable(filter_square_matrix, requires_grad=False)
        
        self.weight = Parameter(torch.Tensor(out_features, in_features))
        if bias:
            self.bias = Parameter(torch.Tensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)
#         print(self.weight.data)
#         print(self.bias.data)

    def forward(self, input):
        return F.linear(input, self.filter_square_matrix.matmul(self.weight.double()), self.bias)

    def __repr__(self):
        return self.__class__.__name__ + '(' \
            + 'in_features=' + str(self.in_features) \
            + ', out_features=' + str(self.out_features) \
            + ', bias=' + str(self.bias is not None) + ')'

In [21]:
class GCN(nn.Module):
      def __init__(self, A, BNP, feature_size, Clamp_A = True, output_last = True):
         super(GCN, self).__init__()
         self.feature_size = feature_size
         self.A = A
         self.BNP = BNP
         self.linear = FilterLinear(410, 410, self.A, bias= False)

      def forward(self, input):
      #some squeeze and unsqueeze
        
          x = torch.mul(self.A, torch.Tensor(self.BNP).double())
          x = self.linear(x)
          x = torch.matmul(x, input.double())
          return x

TSM Modules

```TSM_first``` for the first TSM module and ```TSM_others``` for the remaining modules.

In [22]:
device = 'cpu'
class TSM_first(nn.Module):
  def __init__(self,A, in_features= 200, out_features=200 ):
    super(TSM_first,self).__init__()
    self.GDCN1 = GDCN()
    self.GDCN2 = GDCN()
    self.gat = GAT(in_features= in_features, out_features= out_features)
    self.A = A
    self.GCN1 = GCN(A, BNP, in_features)
    self.GCN2 = GCN(A, BNP, in_features)
    self.BN1 = nn.BatchNorm1d(in_features)
    self.BN2 = nn.BatchNorm1d(in_features)

    self.nodevec1 = nn.Parameter(torch.randn(int(A.shape[0]), 10).to(device), requires_grad=True).to(device)
    self.nodevec2 = nn.Parameter(torch.randn(10, int(A.shape[0])).to(device), requires_grad=True).to(device)

    self.nodevec3 = nn.Parameter(torch.randn(int(A.shape[0]), 10).to(device), requires_grad=True).to(device)
    self.nodevec4 = nn.Parameter(torch.randn(10, int(A.shape[0])).to(device), requires_grad=True).to(device)

  def forward(self,input):
    x = input.reshape((input.shape[0]*input.shape[1]), input.shape[2])
    x1 = self.GDCN1(x)
    x_middle = self.gat(x1,self.A)
    x_middle = self.GCN1(x_middle)
    x = self.BN1(x.permute(1,0)+x_middle.float()).permute(1,0)

    x2 = self.GDCN2(x)
    x_middle = self.gat(x2,self.A)
    x_middle = self.GCN2(x_middle)
    x = self.BN2(x.permute(1,0)+x_middle.float()).permute(1,0)

    return x,x1,x2

In [23]:
class TSM_others(nn.Module):
  def __init__(self,A, in_features= 200, out_features=200 ):
    super(TSM_others,self).__init__()
    self.GDCN1 = GDCN()
    self.GDCN2 = GDCN()
    self.gat = GAT(in_features= in_features, out_features=out_features )
    self.A = A
    self.GCN1 = GCN(A, BNP, in_features)
    self.GCN2 = GCN(A, BNP, in_features)
    self.BN1 = nn.BatchNorm1d(in_features)
    self.BN2 = nn.BatchNorm1d(in_features)

    self.nodevec1 = nn.Parameter(torch.randn(int(A.shape[0]), 10).to(device), requires_grad=True).to(device)
    self.nodevec2 = nn.Parameter(torch.randn(10, int(A.shape[0])).to(device), requires_grad=True).to(device)

    self.nodevec3 = nn.Parameter(torch.randn(int(A.shape[0]), 10).to(device), requires_grad=True).to(device)
    self.nodevec4 = nn.Parameter(torch.randn(10, int(A.shape[0])).to(device), requires_grad=True).to(device)

  def forward(self,x):
    x1 = self.GDCN1(x)
    x_middle = self.gat(x1,self.A)
    x_middle = self.GCN1(x_middle)
    x = self.BN1(x.permute(1,0)+x_middle.float()).permute(1,0)

    x2 = self.GDCN2(x)
    x_middle = self.gat(x2,self.A)
    x_middle = self.GCN2(x_middle)
    x = self.BN2(x.permute(1,0)+x_middle.float()).permute(1,0)

    return x,x1,x2

In [24]:
class linear(nn.Module):
    def __init__(self,c_in,c_out):
        super(linear,self).__init__()
        self.mlp = torch.nn.Conv2d(c_in, c_out, kernel_size=(1, 1), padding=(0,0), stride=(1,1), bias=True)

    def forward(self,x):
        return self.mlp(x)


A component contains the 4 TSM modules, for each type of data

In [25]:
class COMPONENT(nn.Module):
  def __init__(self,A, inp = 200, out=200):
    super(COMPONENT,self).__init__()
    self.A = A
    self.TSM1 = TSM_first(A= torch.tensor(self.A),in_features= inp,out_features=out)    
    self.TSM2 = TSM_others(A= torch.tensor(self.A), in_features= inp,out_features=out)    
    self.TSM3 = TSM_others(A= torch.tensor(self.A),in_features= inp,out_features=out)    
    self.TSM4 = TSM_others(A= torch.tensor(self.A),in_features= inp,out_features=out)
    self.RL = nn.ReLU()
    self.conv1 = linear(1,1)
    self.conv2 = linear(1,1)
    self.linear = nn.Linear(inp, out_features=20)
    

  def forward(self,x):
    x, x11,x12 = self.TSM1(x)
    x, x21,x22 = self.TSM2(x)
    x, x31,x32 = self.TSM3(x)
    x, x41,x42 = self.TSM4(x)

    x = self.RL(x11+x12+x21+x22+x31+x32+x41+x42)
    x = torch.unsqueeze(torch.unsqueeze(x,0),0)
    x = self.conv1(x)
    x = self.RL(x)
    x = self.conv2(x)
    x = self.RL(x)
    x = torch.squeeze(x)
    x = self.linear(x)
    return x.permute(1,0)

## Main model

In [26]:
class Multi_STGAC(nn.Module):
  def __init__(self, A): 
    super(Multi_STGAC,self).__init__()
    self.A=A
    self.Recent_component = COMPONENT(A,20*10,20*10) # 20 is the batch size and 10 is the sequence lenght
    self.Daily_component = COMPONENT(A,20*4,20*4)
    self.Weekly_component = COMPONENT(A,20*2,20*2)
    self.conv1 = nn.Conv2d(in_channels=3*410,
                                    out_channels=410,
                                    kernel_size=1, padding='same')
    self.conv2 = nn.Conv2d(in_channels=410,
                                    out_channels=410,
                                    kernel_size=(1,1), padding='same')
    self.elu = nn.ELU()
  def forward(self,Xr,Xd,Xw):
    X1 = self.Recent_component(Xr)
    X2 = self.Daily_component(Xd)
    X3 = self.Weekly_component(Xw)
    #Concatenation operation
    Y = torch.cat((X1,X2,X3),1)
    Y = torch.unsqueeze(torch.unsqueeze(Y,2),3)
    Y = self.conv1(Y)
    Y = self.elu(Y)
    Y = self.conv2(Y)
    Y = torch.squeeze(Y)
    return Y

## Execute data preparation

In [27]:
train_loader, valid_loader, test_loader, max_load = PrepareDataset(main_df)

### Train model function

In [28]:
def TrainModel(model, train_dataloader, valid_dataloader, learning_rate = 1e-5, num_epochs = 300, patience = 10, min_delta = 0.00001):
    
    
    loss_MSE = torch.nn.MSELoss()
    loss_L1 = torch.nn.L1Loss()

    learning_rate = 1e-5
    optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
        
    interval = 100
    losses_train = []
    losses_valid = []
    losses_epochs_train = []
    losses_epochs_valid = []
    
    cur_time = time.time()
    pre_time = time.time()
    
    # Variables for Early Stopping
    is_best_model = 0
    patient_epoch = 0
    
    for epoch in range(num_epochs):
     
        trained_number = 0
        
        valid_dataloader_iter = iter(valid_dataloader)
        
        losses_epoch_train = []
        losses_epoch_valid = []
        for data in train_dataloader:
            
            inputs_R, labels_R = data[0]
            inputs_D, labels_D = data[1]
            inputs_W, labels_W = data[2]

            inputs_R, labels_R = Variable(inputs_R), Variable(labels_R)
            inputs_D, labels_D = Variable(inputs_D), Variable(labels_D)
            inputs_W, labels_W = Variable(inputs_W), Variable(labels_W)
                
            model.zero_grad()

            outputs = model(inputs_R,inputs_D,inputs_W)
            
            loss_train = loss_MSE(outputs, torch.squeeze(labels_R))
            
            losses_train.append(loss_train.data)
            losses_epoch_train.append(loss_train.data)
            
            optimizer.zero_grad()
            
            loss_train.backward()
            
            optimizer.step()
            
            # validation 
            try: 
                data_val = next(valid_dataloader_iter)
                inputs_val_R, labels_val_R = data_val[0]
                inputs_val_D, labels_val_D = data_val[1]
                inputs_val_W, labels_val_W = data_val[2]

            except StopIteration:
                valid_dataloader_iter = iter(valid_dataloader)
                data_val = next(valid_dataloader_iter)
                
            
            inputs_val_R, labels_val_R = Variable(inputs_val_R), Variable(labels_val_R)
            inputs_val_D, labels_val_D = Variable(inputs_val_D), Variable(labels_val_D)
            inputs_val_W, labels_val_W = Variable(inputs_val_W), Variable(labels_val_W)

            outputs_val= model(inputs_val_R,inputs_val_D,inputs_val_W)

            loss_valid = loss_MSE(outputs_val, torch.squeeze(labels_val_R))
            losses_valid.append(loss_valid.data)
            losses_epoch_valid.append(loss_valid.data)
            
            # output
            trained_number += 1
        
        avg_losses_epoch_train = sum(losses_epoch_train) / float(len(losses_epoch_train))
        avg_losses_epoch_valid = sum(losses_epoch_valid) / float(len(losses_epoch_valid))
        losses_epochs_train.append(avg_losses_epoch_train)
        losses_epochs_valid.append(avg_losses_epoch_valid)
        
        # Early Stopping
        if epoch == 0:
            is_best_model = 1
            best_model = model
            min_loss_epoch_valid = 10000.0
            if avg_losses_epoch_valid < min_loss_epoch_valid:
                min_loss_epoch_valid = avg_losses_epoch_valid
        else:
            if min_loss_epoch_valid - avg_losses_epoch_valid > min_delta:
                is_best_model = 1
                best_model = model
                min_loss_epoch_valid = avg_losses_epoch_valid 
                patient_epoch = 0
            else:
                is_best_model = 0
                patient_epoch += 1
                if patient_epoch >= patience:
                    print('Early Stopped at Epoch:', epoch)
                    break
        
        # Print training parameters
        cur_time = time.time()
        print('Epoch: {}, train_loss: {}, val_loss:{} ,time: {}, best model: {}'.format( \
                    epoch, \
                    np.around(avg_losses_epoch_train, decimals=8),\
                    np.around(avg_losses_epoch_valid, decimals=8),\
                    np.around([cur_time - pre_time] , decimals=2),\
                    is_best_model) )
        pre_time = cur_time
    
    return model, [losses_train, losses_epochs_train, losses_epoch_valid], is_best_model

### Execute and test model

In [29]:
model = Multi_STGAC(torch.tensor(A))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().

In [ ]:
model, [losses_train, losses_epochs_train, losses_epoch_valid], is_best_model = TrainModel(model, train_loader, valid_loader)

In [31]:
def rmse(input,target):
  #input and target of shape (x,y)
  N = target.shape[1]
  return np.sqrt(((input - target)**2).sum(axis=1)/N)

def mae(input,target):
  #input and target of shape (x,y)
  N = target.shape[1]
  return np.abs(input - target).sum(axis=1)/N

# for the 4-bin accuracy
def ranging(table):
  return 1*(table>=0)*(table<=5) + 2*(table>=6)*(table<=10) + 3*(table>=11)*(table<=15)+ 4*(table>=16)

def bins_accuracy(output,label):
  output = ranging(output)
  label = ranging(label)
  return accuracy_score(output,label)

In [32]:
def TestModel(model, test_dataloader, max_load):
    
    cur_time = time.time()
    pre_time = time.time()
    
    use_gpu = torch.cuda.is_available()
    
    
    tested_batch = 0
    
    losses_rmse = []
    losses_mae = []
    acc = []
    acc_bin = []
    
    
    for data in test_dataloader:
        inputs_R, labels_R = data[0]
        inputs_D, labels_D = data[1]
        inputs_W, labels_W = data[2]

        inputs_R, labels_R = Variable(inputs_R), Variable(labels_R)
        inputs_D, labels_D = Variable(inputs_D), Variable(labels_D)
        inputs_W, labels_W = Variable(inputs_W), Variable(labels_W)

        
        outputs =  model(inputs_R,inputs_D,inputs_W)
    
    
        loss_mae = mae(outputs.detach().numpy()*max_load, torch.squeeze(labels_R).detach().numpy()*max_load)
        losses_mae.append(loss_mae)

        loss_rmse = rmse(outputs.detach().numpy()*max_load,torch.squeeze(labels_R).detach().numpy()*max_load)
        losses_rmse.append(loss_rmse)

        accuracy = accuracy_score(np.rint(outputs.detach().numpy().flatten()*max_load),np.rint(torch.squeeze(labels_R).detach().numpy().flatten()*max_load))
        acc.append(accuracy)

        accuracy_bins = bins_accuracy(np.rint(outputs.detach().numpy().flatten()*max_load),np.rint(torch.squeeze(labels_R).detach().numpy().flatten()*max_load))
        acc_bin.append(accuracy_bins)

    mean_mae = np.mean(losses_mae)
    mean_rmse = np.mean(losses_rmse)
    mean_acc = np.mean(acc)
    mean_acc_bin = np.mean(acc_bin)

    print('Tested: mae_mean: {}'.format(mean_mae))
    print('Tested: rmse_mean: {}'.format(mean_rmse))
    print('Tested: accuracy_mean: {}'.format(mean_acc))
    print('Tested: accuracy_bin_mean: {}'.format(mean_acc_bin))
    return  None

In [ ]:
TestModel(model, test_loader,max_load)